In [1]:
import pandas as pd
import csv
from datetime import datetime, timedelta
import os

In [2]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [3]:
'''
For pre-processing, use this tool:
https://docs.google.com/spreadsheets/d/1KP_H4RBrvq-QHJ52Ru_XcD4ZxATtJ1qMr3xfqxVEJVc/edit#gid=2124103360

Instructions after using the UserFilterTool to generate CSVs:
1. Put input CSVs inside the directory specified in the input_folder variable
2. Change the date_subject to the date today. Follow the format YYYYMMDD
3. Run the notebook
4. Get the output from the output folder
5. Upload the files to https://drive.google.com/drive/folders/1QelvFeULpkytJxkWF8XnBOpK7sV6Pk80

'''

date_subject = 20201205

input_folder = 'Sources-Step13/'
lookup_folder = 'Lookup Files/'
output_folder = 'For Validation/'


In [4]:
d7 = pd.read_csv(lookup_folder+"7day.csv")
d30 = pd.read_csv(lookup_folder+"30day.csv")
d90 = pd.read_csv(lookup_folder+"90day.csv")
d365 = pd.read_csv(lookup_folder+"365day.csv")

from functools import reduce
window_date_lookup = reduce(lambda left,right: pd.merge(left,right,on='date'), [d7, d30, d90, d365])
window_date_lookup.columns = ['date', '7d', '30d', '90d', '365d']
window_date_lookup['date'] = pd.to_datetime(window_date_lookup['date']) 
window_date_lookup.set_index('date', inplace=True)
window_date_lookup

,7d,30d,90d,365d
date,,,,
2019-05-27,1,898,1795,2692
2019-05-28,2,899,1796,2693
2019-05-29,3,900,1797,2694
2019-05-30,4,901,1798,2695
2019-05-31,5,902,1799,2696
...,...,...,...,...
2019-04-03,8194,8201,8208,8215
2019-04-04,8195,8202,8209,8216
2019-04-05,8196,8203,8210,8217


In [5]:
country_lookup = pd.read_csv(lookup_folder+"aa_countries.csv")

country_lookup=country_lookup.append({'country':'jfp', 'alpha3_code':'GGG'}, ignore_index=True)
country_lookup=country_lookup.append({'country':'unknown', 'alpha3_code':'GGG'}, ignore_index=True )
country_lookup=country_lookup.append({'country':'russia', 'alpha3_code':'RUS'}, ignore_index=True )
country_lookup=country_lookup.append({'country':'slovakia', 'alpha3_code':'SVK'}, ignore_index=True )
country_lookup.set_index('country', inplace=True)
country_lookup

,alpha3_code
country,
afghanistan,AFG
aland islands,ALA
albania,ALB
algeria,DZA
american samoa,ASM
...,...
vietnam,VNM
jfp,GGG
unknown,GGG


In [6]:
team_lookup = pd.read_csv(lookup_folder+"teams.csv")
team_lookup.name = [str(x).lower() for x in team_lookup.name]
team_lookup.set_index('name', inplace=True)
team_lookup



,id
name,
global staff women,1
global staffweb,2
ldhr,3
leader impact,4
us - legacy ministry,5
global,6
jfp,7
unknown,8


In [7]:
source_directory = input_folder
output_filename_6 = output_folder+str(date_subject)+'_step13.csv'
for root, dirs, files in os.walk(source_directory, topdown=True):
    dirs[:] = [d for d in dirs if d not in exclude]
cpt = len(files)
cpt

1

In [8]:
dfs = []

for entry in files:
    try:
        input_filename_6 = source_directory+'/'+entry

        df6 = pd.read_csv(input_filename_6, header=None)

        df6.columns = ['date','country','7d','30d','90d']
        df6['source_id'] = 87
        df6 = df6.melt(id_vars=['date', 'source_id', 'country'],var_name = 'window', value_name = 'value')
        df6['date'] =  pd.to_datetime(df6['date']) 
        df6.country = [str(x).lower() for x in df6.country]

        df6['isOriginal'] = 1
        df6['activitywindow_date_id'] = window_date_lookup.lookup(df6.date, df6.window)
        
        temp=df6.join(team_lookup, on='country')
        temp['team_id'] = temp.id
        temp.drop(columns=['id'], inplace=True)
        df6 = temp
        
        temp=df6.join(country_lookup, on='country')
        temp.country = temp.alpha3_code
        temp.drop(columns=['alpha3_code'], inplace=True)
        temp = temp.dropna(subset=['country']) 
        df6 = temp
        
        

        final_step6 = df6.copy()

        final_step6['record_id'] = ''
        final_step6.drop(columns=['date', 'window'], inplace=True)
        final_step6 = movecol(final_step6, 
                     cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal', 'team_id'], 
                     ref_col='record_id',
                     place='After')
        
        dfs.append(final_step6)
    except:
        print('Error ' + input_filename_6)

df = pd.concat(dfs)


df.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
df


    

,record_id,activitywindow_date_id,source_id,country,value,isOriginal,team_id
0,,559,87,AUS,0,1,NaN
1,,559,87,CAN,0,1,NaN
2,,559,87,EGY,0,1,NaN
3,,559,87,ETH,0,1,NaN
4,,559,87,FRA,1,1,NaN
5,,559,87,GGG,4,1,6.0
6,,559,87,IND,0,1,NaN
7,,559,87,GGG,1,1,7.0
8,,559,87,LVA,0,1,NaN
9,,559,87,MKD,0,1,NaN
